### This file is for generating actionable insights from the model outputs
- for the sake of creating an MVP im going to use the data directly from dad's store
- For the real product I will pull the data from the models and use those

In [2]:
import pandas as pd
import numpy as np
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/output_data/parts_data.feather")

df['days_of_inventory_outstanding'] = (np.where(df['annual_turnover'] == 0, (df['months_no_sale'] * 30), 365/ df['annual_turnover'])).astype(int)

df['sell_through_rate'] = (np.where(df['quantity'] == 0, 0, (df['rolling_12_month_sales'] / df['quantity'] * 100))).astype(int)

total_sales = []


df[['part_number', 'description', 'quantity', 'price', 'months_no_sale','annual_turnover','days_of_inventory_outstanding', 'annual_days_supply', 'sell_through_rate', 'seasonal_component', 'rolling_12_month_sales']].sort_values(by='days_of_inventory_outstanding', ascending=True)

,part_number,description,quantity,price,months_no_sale,annual_turnover,days_of_inventory_outstanding,annual_days_supply,sell_through_rate,seasonal_component,rolling_12_month_sales
2001,27-l411,dot 3 & 4 (500ml) brk fluid,1,15.99,10,73.0,5,5,6700,67.0,67
5138,504197,sc1 silicone detailer 12oz,1,20.99,4,28.1,12,13,2900,29.0,29
8227,83132467961,"brake fluid dot4 lv, low vis",10,13.99,2,18.2,20,20,1850,211.0,185
5520,5414830,"seal-fuel,pfa",1,57.99,2,17.4,20,21,1700,24.0,17
8379,85040521a,clip,1,0.99,11,15.9,22,23,1600,16.0,16
...,...,...,...,...,...,...,...,...,...,...,...
6707,7119933060,spring washer,15,0.99,7,0.1,3650,5475,6,1.0,1
2014,27-s1210,s100 detailing set,7,18.99,4,0.1,3650,2555,14,1.0,1
8596,88640431a,o-ring,17,0.99,3,0.1,3650,3102,11,3.0,2
6416,7045048,"spring-compr,lp,side mnt,18od",8,14.99,4,0.1,3650,2920,12,2.0,1


In [ ]:
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/output_data/parts_data.feather")
df.columns

Index(['part_number', 'description', 'supplier_name', 'quantity', 'price',
       'margin', 'months_no_sale', 'quantity_ordered_ytd',
       'special_orders_ytd', 'negative_on_hand', 'rolling_12_month_sales',
       'rolling_3_month_sales', 'cost_per_unit', 'total_cost', 'sales_revenue',
       'cogs', 'margin_percentage', 'gross_profit', 'roi',
       'annual_days_supply', 'three_month_days_supply',
       'one_month_days_supply', 'annual_turnover', 'three_month_turnover',
       'one_month_turnover', 'sell_through_rate',
       'days_of_inventory_outstanding', 'order_to_sales_ratio',
       'seasonal_component', 'demand', 'inventory_category', 'safety_stock',
       'reorder_point', 'obsolescence_risk'],
      dtype='object')

In [ ]:
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/Data/Processed/parts_data.feather")
#need to do this for all parts

months = ['sales_last_jan', 'sales_last_feb',
       'sales_last_mar', 'sales_last_apr', 'sales_last_may', 'sales_last_jun',
       'sales_last_jul', 'sales_last_aug', 'sales_last_sep', 'sales_last_oct',
       'sales_last_nov', 'sales_last_dec', 'sales_jan', 'sales_feb']

df['total_sales'] = df[months].sum(axis=1)

# Subtract the seasonal component from the total sales to get seasonally adjusted sales
df['seasonally_adjusted_sales'] = df['total_sales']- df['seasonal_component']

# Select and display the required columns
result = df[['part_number', 'quantity', 'price', 'seasonal_component', 'seasonally_adjusted_sales', 'total_sales']]
result.sort_values(by='total_sales', ascending=False)

,part_number,quantity,price,seasonal_component,seasonally_adjusted_sales,total_sales
7280,540,196,19.49,1235.0,0.0,1235
618,11-282s,0,0.61,1010.0,0.0,1010
1659,153307-l,107,18.99,790.0,0.0,790
7409,550047150,211,21.99,579.0,0.0,579
10853,83212365958,368,20.99,544.0,0.0,544
...,...,...,...,...,...,...
6908,51168547454,2,175.49,0.0,0.0,0
6910,51168557885,2,515.99,0.0,0.0,0
6911,51168557886,1,530.99,0.0,0.0,0
6917,51169151916,74,1.99,0.0,0.0,0


In [ ]:
mask = (df['supplier_name'] == 'bmw group canada') & (df['inventory_category'] == 'obsolete')

obsolete = (df[mask]['quantity']).sum()

total = df['quantity'].sum()

perc = (obsolete / total) * 100

print(f"Obsolete BMW Percentage: {perc}% ")


Obsolete BMW Percentage: 19.100238260510665% 


In [ ]:
import pandas as pd
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/Data/Output/parts_data.feather")

# "what brand has the least total obsolete parts out of my top 5 brands with the highest gross profit?"


mask = df['inventory_category'] == 'obsolete'
qdf = df[mask].groupby('supplier_name')['quantity'].sum().sort_values()

# Calculating total gross profit per supplier
gpdf = df.groupby('supplier_name')['gross_profit'].sum().sort_values(ascending=False)

# Merging the two dataframes
fdf = pd.merge(gpdf, qdf, on='supplier_name', how='left').reset_index()

# Filter for top 5 brands by gross profit
top_5_gross_profit = fdf.nlargest(5, 'gross_profit')

# Sorting the top 5 by quantity of obsolete parts
result = top_5_gross_profit.sort_values(by='quantity', ascending=True)
result

,supplier_name,gross_profit,quantity
4,kimpex,25164.16,272.0
1,parts canada/power twins,57019.86,595.0
3,triumph motorcycles,27687.13,1418.0
2,polaris acceptance,39340.29,2808.0
0,bmw group canada,89368.96,10317.0


In [ ]:
df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/Data/Output/parts_data.feather")
gm = df[['part_number', 'description', 'supplier_name', 'roi']]

# Group by supplier_name and calculate the average margin_percentage
grouped_gm = gm.groupby('supplier_name')['roi'].mean().reset_index()

# Sort by margin_percentage in descending order
grouped_gm_sorted = grouped_gm.sort_values(by='roi', ascending=False)

# Display the result
print(grouped_gm_sorted)

                     supplier_name         roi
17         earthtrack supply group  419.920000
49                    shell canada    9.285000
15                      denali usa    0.000000
59                zero motorcycles  -35.294118
11                bumper to bumper  -44.947143
31                          kimpex  -46.463713
24        harbour city motorsports  -46.780000
43        parts canada/power twins  -47.277031
35                         mavrick  -48.418571
16                          ducati  -48.998689
2                     amazon order  -50.000000
14         cycle works motorsports  -58.120000
42                 out of the blue  -65.163077
21                gamma sales inc.  -65.256154
30                              jb  -66.666667
45              polaris acceptance  -76.887930
38                            moto  -79.087222
4                     beeline moto  -79.887500
53                         triumph  -80.255518
0      action motorcycles / barnes  -81.250000
52           

In [ ]:
obs_cost = df[mask].groupby('supplier_name')['total_cost'].sum()

gp = df.groupby('supplier_name')['gross_profit'].sum()

gm = df.groupby('supplier_name')['margin_percentage'].mean().reset_index()

fdf = pd.merge(obs_cost, gp, on='supplier_name', how='left').reset_index()

df = pd.merge(fdf, gm, on='supplier_name', how='left').reset_index()

df['ratio'] = df['gross_profit']/fdf['total_cost']
df.sort_values(by='ratio', ascending=False)

,index,supplier_name,total_cost,gross_profit,margin_percentage,ratio
4,4,beeline moto,122.40,330.22,57.932500,2.697876
37,37,parts canada/power twins,40359.03,57019.86,44.268908,1.412815
29,29,mavrick,556.74,713.32,48.610000,1.281244
25,25,kimpex,20662.48,25164.16,43.198515,1.217867
16,16,fundy textile and design ltd,633.23,743.13,44.843158,1.173555
17,17,gamma sales inc.,386.07,336.87,36.549231,0.872562
36,36,out of the blue,315.50,130.40,31.155385,0.413312
46,46,triumph,63102.00,24138.48,46.355925,0.382531
22,22,island bmw,1920.11,662.27,42.367000,0.344913
7,7,bmw group canada,267516.74,89368.96,38.642996,0.334069


In [ ]:
import calendar
from datetime import datetime
import pandas as pd

og_df = pd.read_feather("/Users/skylerwilson/Desktop/PartsWise/Data/Processed/parts_data.feather")

def prepare_and_melt_sales_data(df):
    """
    Prepare and melt sales data from wide to long format.

    This function processes a DataFrame containing monthly sales data for two consecutive years. It separates the data for
    the current year and the previous year, melts them into a long format, and then combines the results into a single
    DataFrame. The function also removes the original monthly sales columns from the input DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame containing sales data for the current and previous years. The DataFrame should
                           include columns named 'sales_<month_abbr>' for the current year's sales and 'sales_last_<month_abbr>'
                           for the previous year's sales.

    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: The original DataFrame with the monthly sales columns removed.
            - pd.DataFrame: The melted DataFrame with columns for 'part_number', 'month', 'quantity_sold', and 'year'.
    """
    # Make a copy of the original DataFrame
    df_copy = df.copy()

    # Define the current year and last year for labeling purposes
    current_year = datetime.now().year
    last_year = current_year - 1
    current_month_index = 2 #datetime.now().month <- set to two bc data is from february

    # Define sales columns for this year up to the current month (not including) and all of last year
    this_year_sales_columns = [f'sales_{calendar.month_abbr[i].lower()}' for i in range(1,current_month_index + 1)]
    last_year_sales_columns = [f'sales_last_{calendar.month_abbr[i].lower()}' for i in range(1, 13)]

    # Melt this year's sales data
    df_this_year = pd.melt(df_copy, id_vars=['part_number'], value_vars=this_year_sales_columns, var_name='month', value_name='quantity_sold')
    df_this_year['year'] = current_year

    # Melt last year's sales data
    df_last_year = pd.melt(df_copy, id_vars=['part_number'], value_vars=last_year_sales_columns, var_name='month', value_name='quantity_sold')
    df_last_year['year'] = last_year

    # Map month abbreviations to full names
    month_mapping = {calendar.month_abbr[i].lower(): calendar.month_name[i] for i in range(1, 13)}
    df_this_year['month'] = df_this_year['month'].str.replace('sales_', '').map(month_mapping)
    df_last_year['month'] = df_last_year['month'].str.replace('sales_last_', '').map(month_mapping)

    # Combine the melted data for both years
    df_melted_sales = pd.concat([df_this_year, df_last_year], ignore_index=True)

    monthly_columns_to_drop = this_year_sales_columns + last_year_sales_columns
    df = df.drop(columns=monthly_columns_to_drop)
    return df, df_melted_sales

df, df_melted_sales = prepare_and_melt_sales_data(og_df)
print(df_melted_sales.columns)




Index(['part_number', 'month', 'quantity_sold', 'year'], dtype='object')


In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

def apply_holt_winters(df):
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'], format='%Y-%B')
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    df['forecast'] = pd.NA

    for name, group in df.groupby('part_number'):
        if group['quantity_sold'].sum() == 0:
            df.loc[df['part_number'] == name, 'forecast'] = 0
        else:
            non_zero_data = group[group['quantity_sold'] > 0]
            if non_zero_data['quantity_sold'].nunique() > 1 and len(non_zero_data) >= 3:
                try:
                    model = ExponentialSmoothing(
                        non_zero_data['quantity_sold'],
                        seasonal='add',
                        seasonal_periods=3,
                        trend='add',
                        initialization_method='estimated',
                        use_boxcox=True
                    ).fit()
                    forecast_value = model.forecast(1).iloc[0]
                    df.loc[(df['part_number'] == name) & (df.index == group.index.max()), 'forecast'] = forecast_value
                except Exception as e:
                    df.loc[(df['part_number'] == name) & (df.index == group.index.max()), 'forecast'] = non_zero_data['quantity_sold'].mean()
            else:
                df.loc[(df['part_number'] == name) & (df.index == group.index.max()), 'forecast'] = non_zero_data['quantity_sold'].mean()

    return df

# Example usage
forecast_results = apply_holt_winters(df_melted_sales)
print(forecast_results[['part_number', 'forecast']])



/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index h

            part_number  forecast
date                             
2023-01-01  61212305050         0
2023-01-01     353-0340      <NA>
2023-01-01  35218530611         0
2023-01-01  35218529841      <NA>
2023-01-01  35218523356         0
...                 ...       ...
2024-02-01     353-0654         0
2024-02-01     353-1057  1.187125
2024-02-01     353-3578         0
2024-02-01      3514795         0
2024-02-01    mg6006010         0

[218288 rows x 2 columns]


/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/skylerwilson/miniconda3/envs/partsmatch/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:

mask = forecast_results['forecast'].isna()  # This will correctly identify rows where 'forecast' is NA

# Use the mask to filter the DataFrame and then select the desired columns
filtered_results = forecast_results[mask][['part_number', 'forecast', 'quantity_sold']]

# Optionally, sort the filtered results by 'quantity_sold' if needed
sorted_results = filtered_results.sort_values(by='quantity_sold', ascending=False)

sorted_results


,part_number,forecast,quantity_sold
date,,,
2023-07-01,540,<NA>,196
2023-08-01,540,<NA>,179
2023-03-01,540,<NA>,167
2023-06-01,11-282s,<NA>,146
2023-05-01,83212365958,<NA>,146
...,...,...,...
2023-05-01,286283803,<NA>,0
2023-05-01,286283802,<NA>,0
2023-05-01,286283709,<NA>,0


In [ ]:


# Use the mask to filter the DataFrame and then select the desired columns
filtered_results = forecast_results[['part_number', 'forecast', 'quantity_sold']]

# Optionally, sort the filtered results by 'quantity_sold' if needed
sorted_results = filtered_results.sort_values(by='forecast', ascending=False)

sorted_results

,part_number,forecast,quantity_sold
date,,,
2024-02-01,540,65.737997,0
2024-02-01,83212365958,29.18584,0
2024-02-01,85040821a,23.917064,0
2024-02-01,77210861a,22.0,0
2024-02-01,66910011e,20.095184,0
...,...,...,...
2024-02-01,13717706414,NaN,0
2024-02-01,13717712317,NaN,0
2024-02-01,33177685599,NaN,0
